# **Aspect based Sentiment Analysis**

In [12]:
import numpy as np # linear algebra
import pandas as pd # data processing
import os
import re

In [23]:
import io
toy_rev= pd.read_csv("Scraped_Car_Review_hyundai.csv")
toy_rev

ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.


In [5]:
toy_rev['review']=toy_rev['Review_Title']+toy_rev['Review']

In [6]:
!pip install -U pip setuptools wheel

!pip install -U spacy

!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 21.2 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.
2023-11-23 16:27:00.238266: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-23 16:27:00.238337: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempt

In [7]:
import spacy
from tqdm import tqdm
nlp = spacy.load('en_core_web_sm')

**AMOD - adjectival modifier**
An adjectival modifier of a Noun is any adjectival phrase that serves to modify the meaning of the Noun

ex - 'Great <--amod-- Car', 'Long <--amod-- range'

In [8]:
txt = 'Terrific incident'
doc = nlp(txt)
spacy.displacy.render(doc,style='dep',jupyter=True)

In [9]:
txt = 'Terrific movie'
doc = nlp(txt)
spacy.displacy.render(doc,style='dep',jupyter=True)

In [10]:
txt = 'It is the worst of all'
doc = nlp(txt)
spacy.displacy.render(doc,style='dep',jupyter=True)

**ADVMOD - adverb modifier**
An adverb modifier of a word is a (non-clausal) adverb or adverb-headed phrase that serves to modify
the meaning of the word

ex - 'Drives --advmod--> well

In [11]:
txt = 'We had a great experience at the restaurant, food was delicious, but the service was kinda bad'
doc = nlp(txt)
spacy.displacy.render(doc,style='dep',jupyter=True)

**XCOMP - open clausal complement**
An open clausal complement (xcomp) of a verb or an adjective is a predicative or clausal complement without its own subject

ex - 'wonderful --xcomp--> drive'

In [12]:
txt =  "wonderful to drive the camry "
doc = nlp(txt)
spacy.displacy.render(doc,style='dep',jupyter=True)

In [13]:
competitors = ['Chevy','chevy','Ford','ford','Nissan','nissan','Honda','honda','Chevrolet','chevrolet','Volkswagen','volkswagen','benz','Benz','Mercedes','mercedes','subaru','Subaru','VW']

In [14]:
aspect_terms = []
comp_terms = []
easpect_terms = []
ecomp_terms = []
enemy = []
for x in tqdm(range(len(toy_rev['review']))):
    amod_pairs = []
    advmod_pairs = []
    compound_pairs = []
    xcomp_pairs = []
    neg_pairs = []
    eamod_pairs = []
    eadvmod_pairs = []
    ecompound_pairs = []
    eneg_pairs = []
    excomp_pairs = []
    enemlist = []
    if len(str(toy_rev['review'][x])) != 0:
        lines = str(toy_rev['review'][x]).replace('*',' ').replace('-',' ').replace('so ',' ').replace('be ',' ').replace('are ',' ').replace('just ',' ').replace('get ','').replace('were ',' ').replace('When ','').replace('when ','').replace('again ',' ').replace('where ','').replace('how ',' ').replace('has ',' ').replace('Here ',' ').replace('here ',' ').replace('now ',' ').replace('see ',' ').replace('why ',' ').split('.')
        for line in lines:
            enem_list = []
            for eny in competitors:
                enem = re.search(eny,line)
                if enem is not None:
                    enem_list.append(enem.group())
            if len(enem_list)==0:
                doc = nlp(line)
                str1=''
                str2=''
                for token in doc:
                    if token.pos_ is 'NOUN':
                        for j in token.lefts:
                            if j.dep_ == 'compound':
                                compound_pairs.append((j.text+' '+token.text,token.text))
                            if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                str1 = j.text+' '+token.text
                                amod_pairs.append(j.text+' '+token.text)
                                for k in j.lefts:
                                    if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                        str2 = k.text+' '+j.text+' '+token.text
                                        amod_pairs.append(k.text+' '+j.text+' '+token.text)
                                mtch = re.search(re.escape(str1),re.escape(str2))
                                if mtch is not None:
                                    amod_pairs.remove(str1)
                    if token.pos_ is 'VERB':
                        for j in token.lefts:
                            if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                advmod_pairs.append(j.text+' '+token.text)
                            if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                neg_pairs.append(j.text+' '+token.text)
                        for j in token.rights:
                            if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                advmod_pairs.append(token.text+' '+j.text)
                    if token.pos_ is 'ADJ':
                        for j,h in zip(token.rights,token.lefts):
                            if j.dep_ is 'xcomp' and h.dep_ is not 'neg':
                                for k in j.lefts:
                                    if k.dep_ is 'aux':
                                        xcomp_pairs.append(token.text+' '+k.text+' '+j.text)
                            elif j.dep_ is 'xcomp' and h.dep_ is 'neg':
                                if k.dep_ is 'aux':
                                        neg_pairs.append(h.text +' '+token.text+' '+k.text+' '+j.text)

            else:
                enemlist.append(enem_list)
                doc = nlp(line)
                str1=''
                str2=''
                for token in doc:
                    if token.pos_ is 'NOUN':
                        for j in token.lefts:
                            if j.dep_ == 'compound':
                                ecompound_pairs.append((j.text+' '+token.text,token.text))
                            if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                str1 = j.text+' '+token.text
                                eamod_pairs.append(j.text+' '+token.text)
                                for k in j.lefts:
                                    if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                        str2 = k.text+' '+j.text+' '+token.text
                                        eamod_pairs.append(k.text+' '+j.text+' '+token.text)
                                mtch = re.search(re.escape(str1),re.escape(str2))
                                if mtch is not None:
                                    eamod_pairs.remove(str1)
                    if token.pos_ is 'VERB':
                        for j in token.lefts:
                            if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                eadvmod_pairs.append(j.text+' '+token.text)
                            if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                eneg_pairs.append(j.text+' '+token.text)
                        for j in token.rights:
                            if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                eadvmod_pairs.append(token.text+' '+j.text)
                    if token.pos_ is 'ADJ':
                        for j in token.rights:
                            if j.dep_ is 'xcomp':
                                for k in j.lefts:
                                    if k.dep_ is 'aux':
                                        excomp_pairs.append(token.text+' '+k.text+' '+j.text)
        pairs = list(set(amod_pairs+advmod_pairs+neg_pairs+xcomp_pairs))
        epairs = list(set(eamod_pairs+eadvmod_pairs+eneg_pairs+excomp_pairs))
        for i in range(len(pairs)):
            if len(compound_pairs)!=0:
                for comp in compound_pairs:
                    mtch = re.search(re.escape(comp[1]),re.escape(pairs[i]))
                    if mtch is not None:
                        pairs[i] = pairs[i].replace(mtch.group(),comp[0])
        for i in range(len(epairs)):
            if len(ecompound_pairs)!=0:
                for comp in ecompound_pairs:
                    mtch = re.search(re.escape(comp[1]),re.escape(epairs[i]))
                    if mtch is not None:
                        epairs[i] = epairs[i].replace(mtch.group(),comp[0])

    aspect_terms.append(pairs)
    comp_terms.append(compound_pairs)
    easpect_terms.append(epairs)
    ecomp_terms.append(ecompound_pairs)
    enemy.append(enemlist)
toy_rev['compound_nouns'] = comp_terms
toy_rev['aspect_keywords'] = aspect_terms
toy_rev['competition'] = enemy
toy_rev['competition_comp_nouns'] = ecomp_terms
toy_rev['competition_aspects'] = easpect_terms
toy_rev.head()

<>:31: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:35: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:35: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:39: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:45: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:47: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:47: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:49: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:49: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:52: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:52: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:54: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:56: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:56: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:58: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:60: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:6

,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating\r,review,compound_nouns,aspect_keywords,competition,competition_comp_nouns,competition_aspects
0,0,on 03/28/18 20:22 PM (PDT),Kiowa1@gmail.com,2006 Hyundai Azera Sedan Limited 4dr Sedan (3....,Excellent Value,I bought the 2006 Azera Limited in June of 20...,4.0,Excellent Value I bought the 2006 Azera Limite...,[],"[negative comment, only comment, rough roads, ...",[],[],[]
1,1,on 05/02/17 18:03 PM (PDT),SiGung,2006 Hyundai Azera Sedan Limited 4dr Sedan (3....,Don't even waste your time looking much less b...,Electrical nightmare. Dealer cost for repair...,1.0,Don't even waste your time looking much less b...,"[(buyin nightmare, nightmare)]","[even waste, Electrical buyin nightmare]",[[Honda]],[],[]
2,2,on 05/02/17 18:03 PM (PDT),SiGung,2006 Hyundai Azera Sedan Limited 4dr Sedan (3....,Great car,"Great car, great ride, would have better mile...",5.0,"Great car Great car, great ride, would have be...","[(speed transmission, transmission)]","[better mileage, Great car, great ride]",[],[],[]
3,3,on 03/27/17 14:49 PM (PDT),Kevin Gibson,2006 Hyundai Azera Sedan Limited 4dr Sedan (3....,Best car ever,I absolutly love my Azera. The performance is...,5.0,Best car ever I absolutly love my Azera. The p...,"[(luxury interior, interior)]","[ever love, Best car, Just go, absolutly love]",[],[],[]
4,4,on 01/10/17 10:22 AM (PST),Nick Casper,2006 Hyundai Azera Sedan Limited 4dr Sedan (3....,Good value for luxury,We have had our 2006 Azera from nearly new (3...,4.0,Good value for luxury We have had our 2006 Aze...,"[(fuel economy, economy)]","[great car, Good value, new miles, nearly new ...",[],[],[]


In [15]:
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.7 MB/s eta 0:00:00


In [19]:
import operator
sentiment = []
for i in range(len(toy_rev)):
    score_dict={'pos':0,'neg':0,'neu':0}
    if len(toy_rev['aspect_keywords'][i])!=0:
        for aspects in toy_rev['aspect_keywords'][i]:
            sent = analyser.polarity_scores(aspects)
            score_dict['neg'] += sent['neg']
            score_dict['pos'] += sent['pos']
            #score_dict['neu'] += sent['neu']
        sentiment.append(max(score_dict.items(), key=operator.itemgetter(1))[0])
    else:
        sentiment.append('NaN')
toy_rev['sentiment'] = sentiment
toy_rev.head()

,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating\r,review,compound_nouns,aspect_keywords,competition,competition_comp_nouns,competition_aspects,sentiment,int_sent
0,0,on 03/28/18 20:22 PM (PDT),Kiowa1@gmail.com,2006 Hyundai Azera Sedan Limited 4dr Sedan (3....,Excellent Value,I bought the 2006 Azera Limited in June of 20...,4.0,Excellent Value I bought the 2006 Azera Limite...,[],"[negative comment, only comment, rough roads, ...",[],[],[],pos,1
1,1,on 05/02/17 18:03 PM (PDT),SiGung,2006 Hyundai Azera Sedan Limited 4dr Sedan (3....,Don't even waste your time looking much less b...,Electrical nightmare. Dealer cost for repair...,1.0,Don't even waste your time looking much less b...,"[(buyin nightmare, nightmare)]","[even waste, Electrical buyin nightmare]",[[Honda]],[],[],neg,0
2,2,on 05/02/17 18:03 PM (PDT),SiGung,2006 Hyundai Azera Sedan Limited 4dr Sedan (3....,Great car,"Great car, great ride, would have better mile...",5.0,"Great car Great car, great ride, would have be...","[(speed transmission, transmission)]","[better mileage, Great car, great ride]",[],[],[],pos,1
3,3,on 03/27/17 14:49 PM (PDT),Kevin Gibson,2006 Hyundai Azera Sedan Limited 4dr Sedan (3....,Best car ever,I absolutly love my Azera. The performance is...,5.0,Best car ever I absolutly love my Azera. The p...,"[(luxury interior, interior)]","[ever love, Best car, Just go, absolutly love]",[],[],[],pos,1
4,4,on 01/10/17 10:22 AM (PST),Nick Casper,2006 Hyundai Azera Sedan Limited 4dr Sedan (3....,Good value for luxury,We have had our 2006 Azera from nearly new (3...,4.0,Good value for luxury We have had our 2006 Aze...,"[(fuel economy, economy)]","[great car, Good value, new miles, nearly new ...",[],[],[],pos,1


In [20]:
int_sent = []
for sent in toy_rev['sentiment']:
    if sent is 'NaN':
        int_sent.append('NaN')
    elif sent is 'pos':
        int_sent.append('1')
    else:
        int_sent.append('0')
toy_rev['int_sent'] = int_sent
toy_rev.head()

<>:3: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:5: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:3: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:5: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-20-53d0265b703b>:3: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if sent is 'NaN':
<ipython-input-20-53d0265b703b>:5: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif sent is 'pos':


,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating\r,review,compound_nouns,aspect_keywords,competition,competition_comp_nouns,competition_aspects,sentiment,int_sent
0,0,on 03/28/18 20:22 PM (PDT),Kiowa1@gmail.com,2006 Hyundai Azera Sedan Limited 4dr Sedan (3....,Excellent Value,I bought the 2006 Azera Limited in June of 20...,4.0,Excellent Value I bought the 2006 Azera Limite...,[],"[negative comment, only comment, rough roads, ...",[],[],[],pos,1
1,1,on 05/02/17 18:03 PM (PDT),SiGung,2006 Hyundai Azera Sedan Limited 4dr Sedan (3....,Don't even waste your time looking much less b...,Electrical nightmare. Dealer cost for repair...,1.0,Don't even waste your time looking much less b...,"[(buyin nightmare, nightmare)]","[even waste, Electrical buyin nightmare]",[[Honda]],[],[],neg,0
2,2,on 05/02/17 18:03 PM (PDT),SiGung,2006 Hyundai Azera Sedan Limited 4dr Sedan (3....,Great car,"Great car, great ride, would have better mile...",5.0,"Great car Great car, great ride, would have be...","[(speed transmission, transmission)]","[better mileage, Great car, great ride]",[],[],[],pos,1
3,3,on 03/27/17 14:49 PM (PDT),Kevin Gibson,2006 Hyundai Azera Sedan Limited 4dr Sedan (3....,Best car ever,I absolutly love my Azera. The performance is...,5.0,Best car ever I absolutly love my Azera. The p...,"[(luxury interior, interior)]","[ever love, Best car, Just go, absolutly love]",[],[],[],pos,1
4,4,on 01/10/17 10:22 AM (PST),Nick Casper,2006 Hyundai Azera Sedan Limited 4dr Sedan (3....,Good value for luxury,We have had our 2006 Azera from nearly new (3...,4.0,Good value for luxury We have had our 2006 Aze...,"[(fuel economy, economy)]","[great car, Good value, new miles, nearly new ...",[],[],[],pos,1


In [21]:
import math
pos = []
for i in range(len(toy_rev)):
    if not math.isnan(toy_rev['Rating\r'][i]):
        if int(toy_rev['Rating\r'][i])>3:
            pos.append('1')
        else:
            pos.append('0')
    else:
        pos.append('0')
toy_rev['Positive Review'] = pos
toy_rev.head()

,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating\r,review,compound_nouns,aspect_keywords,competition,competition_comp_nouns,competition_aspects,sentiment,int_sent,Positive Review
0,0,on 03/28/18 20:22 PM (PDT),Kiowa1@gmail.com,2006 Hyundai Azera Sedan Limited 4dr Sedan (3....,Excellent Value,I bought the 2006 Azera Limited in June of 20...,4.0,Excellent Value I bought the 2006 Azera Limite...,[],"[negative comment, only comment, rough roads, ...",[],[],[],pos,1,1
1,1,on 05/02/17 18:03 PM (PDT),SiGung,2006 Hyundai Azera Sedan Limited 4dr Sedan (3....,Don't even waste your time looking much less b...,Electrical nightmare. Dealer cost for repair...,1.0,Don't even waste your time looking much less b...,"[(buyin nightmare, nightmare)]","[even waste, Electrical buyin nightmare]",[[Honda]],[],[],neg,0,0
2,2,on 05/02/17 18:03 PM (PDT),SiGung,2006 Hyundai Azera Sedan Limited 4dr Sedan (3....,Great car,"Great car, great ride, would have better mile...",5.0,"Great car Great car, great ride, would have be...","[(speed transmission, transmission)]","[better mileage, Great car, great ride]",[],[],[],pos,1,1
3,3,on 03/27/17 14:49 PM (PDT),Kevin Gibson,2006 Hyundai Azera Sedan Limited 4dr Sedan (3....,Best car ever,I absolutly love my Azera. The performance is...,5.0,Best car ever I absolutly love my Azera. The p...,"[(luxury interior, interior)]","[ever love, Best car, Just go, absolutly love]",[],[],[],pos,1,1
4,4,on 01/10/17 10:22 AM (PST),Nick Casper,2006 Hyundai Azera Sedan Limited 4dr Sedan (3....,Good value for luxury,We have had our 2006 Azera from nearly new (3...,4.0,Good value for luxury We have had our 2006 Aze...,"[(fuel economy, economy)]","[great car, Good value, new miles, nearly new ...",[],[],[],pos,1,1


In [22]:
d = {'sent':toy_rev['Positive Review'],'sent_pred':toy_rev['int_sent']}
metric_df = pd.DataFrame(data=d)
metric_df.head()

,sent,sent_pred
0,1,1
1,0,0
2,1,1
3,1,1
4,1,1


In [23]:
len(metric_df.sent)

8226

In [24]:
metric_df = metric_df[metric_df.sent_pred != 'NaN']
len(metric_df.sent)

7971

In [25]:
from sklearn.metrics import accuracy_score,auc,f1_score,recall_score,precision_score
print('accuracy')
print(accuracy_score(metric_df.sent, metric_df.sent_pred))
print('f1 score')
print(f1_score(metric_df.sent, metric_df.sent_pred,pos_label='1'))
print('recall')
print(recall_score(metric_df.sent, metric_df.sent_pred,pos_label='1'))
print('precision')
print(precision_score(metric_df.sent, metric_df.sent_pred,pos_label='1'))

accuracy
0.7818341487893614
f1 score
0.8621919327997464
recall
0.8901980036000654
precision
0.8358942839582053


>## Product Review



In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()
text_db = ['problem of evil',
          'evil queen',
          'horizon problem']
tf_idf =  vec.fit_transform(text_db)
# print(pd.DataFrame(tf_idf.toarray(), columns=vec.get_feature_names()))

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory




In [ ]:
from google.colab import files
uploaded = files.upload()

Saving amazon_cells_labelled.csv to amazon_cells_labelled.csv


# **Amazon cells dataset**

In [27]:
import pandas as pd
import io
amazon = pd.read_csv('amazon_cells_labelled.csv')
amazon

,Review,Output
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
...,...,...
995,The screen does get smudged easily because it ...,0
996,What a piece of junk.. I lose more calls on th...,0
997,Item Does Not Match Picture.,0
998,The only thing that disappoint me is the infra...,0


In [28]:
#amazon
list(amazon.columns)

['Review', 'Output']

In [29]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [30]:
import numpy as np # linear algebra
import pandas as pd # data processing
import os
import re
competitors = ['Motorola','Samsung','Apple','Huawei','Nokia','Sony','LG','HTC','Google','OnePlus','Xiaomi','LG','Oppo','Vivo']

aspect_terms = []
comp_terms = []
easpect_terms = []
ecomp_terms = []
enemy = []
for x in tqdm(range(len(amazon['Review']))):
    amod_pairs = []
    advmod_pairs = []
    compound_pairs = []
    xcomp_pairs = []
    neg_pairs = []
    eamod_pairs = []
    eadvmod_pairs = []
    ecompound_pairs = []
    eneg_pairs = []
    excomp_pairs = []
    enemlist = []
    if len(str(amazon['Review'][x])) != 0:
        lines = str(amazon['Review'][x]).replace('*',' ').replace('-',' ').replace('so ',' ').replace('be ',' ').replace('are ',' ').replace('just ',' ').replace('get ','').replace('were ',' ').replace('When ','').replace('when ','').replace('again ',' ').replace('where ','').replace('how ',' ').replace('has ',' ').replace('Here ',' ').replace('here ',' ').replace('now ',' ').replace('see ',' ').replace('why ',' ').split('.')
        for line in lines:
            enem_list = []
            for eny in competitors:
                enem = re.search(eny,line)
                if enem is not None:
                    enem_list.append(enem.group())
            if len(enem_list)==0:
                doc = nlp(line)
                str1=''
                str2=''
                for token in doc:
                    if token.pos_ is 'NOUN':
                        for j in token.lefts:
                            if j.dep_ == 'compound':
                                compound_pairs.append((j.text+' '+token.text,token.text))
                            if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                str1 = j.text+' '+token.text
                                amod_pairs.append(j.text+' '+token.text)
                                for k in j.lefts:
                                    if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                        str2 = k.text+' '+j.text+' '+token.text
                                        amod_pairs.append(k.text+' '+j.text+' '+token.text)
                                mtch = re.search(re.escape(str1),re.escape(str2))
                                if mtch is not None:
                                    amod_pairs.remove(str1)
                    if token.pos_ is 'VERB':
                        for j in token.lefts:
                            if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                advmod_pairs.append(j.text+' '+token.text)
                            if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                neg_pairs.append(j.text+' '+token.text)
                        for j in token.rights:
                            if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                advmod_pairs.append(token.text+' '+j.text)
                    if token.pos_ is 'ADJ':
                        for j,h in zip(token.rights,token.lefts):
                            if j.dep_ is 'xcomp' and h.dep_ is not 'neg':
                                for k in j.lefts:
                                    if k.dep_ is 'aux':
                                        xcomp_pairs.append(token.text+' '+k.text+' '+j.text)
                            elif j.dep_ is 'xcomp' and h.dep_ is 'neg':
                                if k.dep_ is 'aux':
                                        neg_pairs.append(h.text +' '+token.text+' '+k.text+' '+j.text)

            else:
                enemlist.append(enem_list)
                doc = nlp(line)
                str1=''
                str2=''
                for token in doc:
                    if token.pos_ is 'NOUN':
                        for j in token.lefts:
                            if j.dep_ == 'compound':
                                ecompound_pairs.append((j.text+' '+token.text,token.text))
                            if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                str1 = j.text+' '+token.text
                                eamod_pairs.append(j.text+' '+token.text)
                                for k in j.lefts:
                                    if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                        str2 = k.text+' '+j.text+' '+token.text
                                        eamod_pairs.append(k.text+' '+j.text+' '+token.text)
                                mtch = re.search(re.escape(str1),re.escape(str2))
                                if mtch is not None:
                                    eamod_pairs.remove(str1)
                    if token.pos_ is 'VERB':
                        for j in token.lefts:
                            if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                eadvmod_pairs.append(j.text+' '+token.text)
                            if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                eneg_pairs.append(j.text+' '+token.text)
                        for j in token.rights:
                            if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                eadvmod_pairs.append(token.text+' '+j.text)
                    if token.pos_ is 'ADJ':
                        for j in token.rights:
                            if j.dep_ is 'xcomp':
                                for k in j.lefts:
                                    if k.dep_ is 'aux':
                                        excomp_pairs.append(token.text+' '+k.text+' '+j.text)
        pairs = list(set(amod_pairs+advmod_pairs+neg_pairs+xcomp_pairs))
        epairs = list(set(eamod_pairs+eadvmod_pairs+eneg_pairs+excomp_pairs))
        for i in range(len(pairs)):
            if len(compound_pairs)!=0:
                for comp in compound_pairs:
                    mtch = re.search(re.escape(comp[1]),re.escape(pairs[i]))
                    if mtch is not None:
                        pairs[i] = pairs[i].replace(mtch.group(),comp[0])
        for i in range(len(epairs)):
            if len(ecompound_pairs)!=0:
                for comp in ecompound_pairs:
                    mtch = re.search(re.escape(comp[1]),re.escape(epairs[i]))
                    if mtch is not None:
                        epairs[i] = epairs[i].replace(mtch.group(),comp[0])

    aspect_terms.append(pairs)
    comp_terms.append(compound_pairs)
    easpect_terms.append(epairs)
    ecomp_terms.append(ecompound_pairs)
    enemy.append(enemlist)
amazon['compound_nouns'] = comp_terms
amazon['aspect_keywords'] = aspect_terms
amazon['competition'] = enemy
amazon['competition_comp_nouns'] = ecomp_terms
amazon['competition_aspects'] = easpect_terms
amazon.head()

import operator
sentiment = []
# for i in range(len(amazon)):
#     score_dict={'pos':0,'neg':0,'neu':0}
#     if len(amazon['aspect_keywords'][i])!=0:
#         for aspects in amazon['aspect_keywords'][i]:
#             sent = analyser.polarity_scores(aspect)
#             score_dict['neg'] += sent['neg']
#             score_dict['pos'] += sent['pos']
        #score_dict['neu'] += sent['neu']
for i in range(len(amazon['aspect_keywords'])):
    if len(amazon['aspect_keywords'][i]) != 0:
        for aspect in amazon['aspect_keywords'][i]:
            # Ensure 'aspect' is properly initialized within the loop
            sent = analyser.polarity_scores(aspect)
            score_dict['neg'] += sent['neg']
            score_dict['pos'] += sent['pos']
        sentiment.append(max(score_dict.items(), key=operator.itemgetter(1))[0])
    else:
        sentiment.append('NaN')
amazon['sentiment'] = sentiment
amazon.head()

int_sent = []
for sent in amazon['sentiment']:
    # if sent is 'NaN':
    #     int_sent.append('NaN')
    # elif sent is 'pos':
    #     int_sent.append('1')
    # else:
    #     int_sent.append('0')
    if sent is 'neg':
        int_sent.append('0')
    elif sent is 'pos':
        int_sent.append('1')
    else:
        int_sent.append('NaN')
amazon['int_sent'] = int_sent
amazon

<>:37: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:41: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:41: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:45: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:51: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:53: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:53: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:55: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:55: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:58: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:58: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:60: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:62: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:62: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:64: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:66: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:6

,Review,Output,compound_nouns,aspect_keywords,competition,competition_comp_nouns,competition_aspects,sentiment,int_sent
0,So there is no way for me to plug it in here i...,0,[],[],[],[],[],NaN,NaN
1,"Good case, Excellent value.",1,[],"[Excellent value, Good case]",[],[],[],pos,1
2,Great for the jawbone.,1,[],[],[],[],[],NaN,NaN
3,Tied to charger for conversations lasting more...,0,[],[],[],[],[],NaN,NaN
4,The mic is great.,1,[],[],[],[],[],NaN,NaN
...,...,...,...,...,...,...,...,...,...
995,The screen does get smudged easily because it ...,0,[],[smudged easily],[],[],[],pos,1
996,What a piece of junk.. I lose more calls on th...,0,[],[more calls],[],[],[],pos,1
997,Item Does Not Match Picture.,0,[],[],[],[],[],NaN,NaN
998,The only thing that disappoint me is the infra...,0,[],"[infra port, only thing, red port]",[],[],[],pos,1


# **Imdb dataset**

In [31]:
import pandas as pd
import io
imdb = pd.read_csv('imdb_labelled.csv')
imdb

,Review,Output
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
...,...,...
995,I just got bored watching Jessice Lange take h...,0
996,"Unfortunately, any virtue in this film's produ...",0
997,"In a word, it is embarrassing.",0
998,Exceptionally bad!,0


In [32]:
#imdb
list(imdb.columns)

['Review', 'Output']

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
!pip install scikit-learn
from tqdm import tqdm

In [35]:
imdb_train = imdb[:500]
imdb_test = imdb[500:]

vectorizer = TfidfVectorizer(use_idf=True,ngram_range=(1,1))
tfidf_features_train = vectorizer.fit_transform(imdb_train['Review'])
tfidf_features_test = vectorizer.transform(imdb_test['Review'])
print (tfidf_features_train.shape, tfidf_features_test.shape)

(500, 1809) (500, 1809)


In [36]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [37]:
from inspect import ismemberdescriptor
import numpy as np # linear algebra
import pandas as pd # data processing
import os
import re

import spacy

# Load the spaCy English language model
nlp = spacy.load("en_core_web_sm")

# reviews = imdb.get('Review', [])
reviews=imdb['Review']
competitors = []

aspect_terms = []
comp_terms = []
easpect_terms = []
ecomp_terms = []
enemy = []
for x in tqdm(range(len(reviews))):
    review_text = reviews[x]
    amod_pairs = []
    advmod_pairs = []
    compound_pairs = []
    xcomp_pairs = []
    neg_pairs = []
    eamod_pairs = []
    eadvmod_pairs = []
    ecompound_pairs = []
    eneg_pairs = []
    excomp_pairs = []
    enemlist = []
    if len(str(imdb['Review'][x])) != 0:
        lines = str(imdb['Review'][x]).replace('*',' ').replace('-',' ').replace('so ',' ').replace('be ',' ').replace('are ',' ').replace('just ',' ').replace('get ','').replace('were ',' ').replace('When ','').replace('when ','').replace('again ',' ').replace('where ','').replace('how ',' ').replace('has ',' ').replace('Here ',' ').replace('here ',' ').replace('now ',' ').replace('see ',' ').replace('why ',' ').split('.')
        for line in lines:
            enem_list = []
            for eny in competitors:
                enem = re.search(eny,line)
                if enem is not None:
                    enem_list.append(enem.group())
            if len(enem_list)==0:
                doc = nlp(line)
                str1=''
                str2=''
                for token in doc:
                    if token.pos_ is 'NOUN':
                        for j in token.lefts:
                            if j.dep_ == 'compound':
                                compound_pairs.append((j.text+' '+token.text,token.text))
                            if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                str1 = j.text+' '+token.text
                                amod_pairs.append(j.text+' '+token.text)
                                for k in j.lefts:
                                    if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                        str2 = k.text+' '+j.text+' '+token.text
                                        amod_pairs.append(k.text+' '+j.text+' '+token.text)
                                mtch = re.search(re.escape(str1),re.escape(str2))
                                if mtch is not None:
                                    amod_pairs.remove(str1)
                    if token.pos_ is 'VERB':
                        for j in token.lefts:
                            if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                advmod_pairs.append(j.text+' '+token.text)
                            if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                neg_pairs.append(j.text+' '+token.text)
                        for j in token.rights:
                            if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                advmod_pairs.append(token.text+' '+j.text)
                    if token.pos_ is 'ADJ':
                        for j,h in zip(token.rights,token.lefts):
                            if j.dep_ is 'xcomp' and h.dep_ is not 'neg':
                                for k in j.lefts:
                                    if k.dep_ is 'aux':
                                        xcomp_pairs.append(token.text+' '+k.text+' '+j.text)
                            elif j.dep_ is 'xcomp' and h.dep_ is 'neg':
                                if k.dep_ is 'aux':
                                        neg_pairs.append(h.text +' '+token.text+' '+k.text+' '+j.text)

            else:
                enemlist.append(enem_list)
                doc = nlp(line)
                str1=''
                str2=''
                for token in doc:
                    if token.pos_ is 'NOUN':
                        for j in token.lefts:
                            if j.dep_ == 'compound':
                                ecompound_pairs.append((j.text+' '+token.text,token.text))
                            if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                str1 = j.text+' '+token.text
                                eamod_pairs.append(j.text+' '+token.text)
                                for k in j.lefts:
                                    if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                        str2 = k.text+' '+j.text+' '+token.text
                                        eamod_pairs.append(k.text+' '+j.text+' '+token.text)
                                mtch = re.search(re.escape(str1),re.escape(str2))
                                if mtch is not None:
                                    eamod_pairs.remove(str1)
                    if token.pos_ is 'VERB':
                        for j in token.lefts:
                            if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                eadvmod_pairs.append(j.text+' '+token.text)
                            if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                eneg_pairs.append(j.text+' '+token.text)
                        for j in token.rights:
                            if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                eadvmod_pairs.append(token.text+' '+j.text)
                    if token.pos_ is 'ADJ':
                        for j in token.rights:
                            if j.dep_ is 'xcomp':
                                for k in j.lefts:
                                    if k.dep_ is 'aux':
                                        excomp_pairs.append(token.text+' '+k.text+' '+j.text)
        pairs = list(set(amod_pairs+advmod_pairs+neg_pairs+xcomp_pairs))
        epairs = list(set(eamod_pairs+eadvmod_pairs+eneg_pairs+excomp_pairs))
        for i in range(len(pairs)):
            if len(compound_pairs)!=0:
                for comp in compound_pairs:
                    mtch = re.search(re.escape(comp[1]),re.escape(pairs[i]))
                    if mtch is not None:
                        pairs[i] = pairs[i].replace(mtch.group(),comp[0])
        for i in range(len(epairs)):
            if len(ecompound_pairs)!=0:
                for comp in ecompound_pairs:
                    mtch = re.search(re.escape(comp[1]),re.escape(epairs[i]))
                    if mtch is not None:
                        epairs[i] = epairs[i].replace(mtch.group(),comp[0])

    aspect_terms.append(pairs)
    comp_terms.append(compound_pairs)
    easpect_terms.append(epairs)
    ecomp_terms.append(ecompound_pairs)
    enemy.append(enemlist)
imdb['compound_nouns'] = comp_terms
imdb['aspect_keywords'] = aspect_terms
imdb['competition'] = enemy
imdb['competition_comp_nouns'] = ecomp_terms
imdb['competition_aspects'] = easpect_terms
imdb.head()

import operator
sentiment = []
for i in range(len(imdb)):
    score_dict={'pos':0,'neg':0,'neu':0}
    if len(imdb['aspect_keywords'][i])!=0:
        for aspects in imdb['aspect_keywords'][i]:
            sent = analyser.polarity_scores(aspects)
            score_dict['neg'] += sent['neg']
            score_dict['pos'] += sent['pos']
        #score_dict['neu'] += sent['neu']
        sentiment.append(max(score_dict.items(), key=operator.itemgetter(1))[0])
    else:
        sentiment.append('NaN')
imdb['sentiment'] = sentiment
imdb.head()

int_sent = []
for sent in imdb['sentiment']:
    if sent is 'NaN':
        int_sent.append('NaN')
    elif sent is 'pos':
        int_sent.append('1')
    else:
        int_sent.append('0')
imdb['int_sent'] = int_sent
imdb

<>:47: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:51: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:51: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:55: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:61: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:63: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:63: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:65: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:65: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:68: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:68: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:70: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:72: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:72: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:74: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:76: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:7

,Review,Output,compound_nouns,aspect_keywords,competition,competition_comp_nouns,competition_aspects,sentiment,int_sent
0,"A very, very, very slow-moving, aimless movie ...",0,[],"[aimless movie, young man]",[],[],[],pos,1
1,Not sure who was more lost - the flat characte...,0,[],[flat characters],[],[],[],pos,1
2,Attempting artiness with black & white and cle...,0,"[(camera angles, angles)]",[black camera angles],[],[],[],pos,1
3,Very little music or anything to speak of.,0,[],"[little music, Very little music]",[],[],[],pos,1
4,The best scene in the movie was when Gerardo i...,1,[],[best scene],[],[],[],pos,1
...,...,...,...,...,...,...,...,...,...
995,I just got bored watching Jessice Lange take h...,0,[],[],[],[],[],NaN,NaN
996,"Unfortunately, any virtue in this film's produ...",0,"[(production work, work)]","[regrettable script, Unfortunately lost]",[],[],[],neg,0
997,"In a word, it is embarrassing.",0,[],[],[],[],[],NaN,NaN
998,Exceptionally bad!,0,[],[],[],[],[],NaN,NaN


# **Yelp**

In [38]:
import pandas as pd
import io
yelp = pd.read_csv('yelp_labelled.csv')
yelp

,Review,Output
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


In [39]:
list(yelp.columns)

['Review', 'Output']

In [40]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [41]:
from inspect import ismemberdescriptor
import numpy as np # linear algebra
import pandas as pd # data processing
import os
import re
competitors = []

aspect_terms = []
comp_terms = []
easpect_terms = []
ecomp_terms = []
enemy = []
for x in tqdm(range(len(yelp['Review']))):
    amod_pairs = []
    advmod_pairs = []
    compound_pairs = []
    xcomp_pairs = []
    neg_pairs = []
    eamod_pairs = []
    eadvmod_pairs = []
    ecompound_pairs = []
    eneg_pairs = []
    excomp_pairs = []
    enemlist = []
    if len(str(yelp['Review'][x])) != 0:
        lines = str(yelp['Review'][x]).replace('*',' ').replace('-',' ').replace('so ',' ').replace('be ',' ').replace('are ',' ').replace('just ',' ').replace('get ','').replace('were ',' ').replace('When ','').replace('when ','').replace('again ',' ').replace('where ','').replace('how ',' ').replace('has ',' ').replace('Here ',' ').replace('here ',' ').replace('now ',' ').replace('see ',' ').replace('why ',' ').split('.')
        for line in lines:
            enem_list = []
            for eny in competitors:
                enem = re.search(eny,line)
                if enem is not None:
                    enem_list.append(enem.group())
            if len(enem_list)==0:
                doc = nlp(line)
                str1=''
                str2=''
                for token in doc:
                    if token.pos_ is 'NOUN':
                        for j in token.lefts:
                            if j.dep_ == 'compound':
                                compound_pairs.append((j.text+' '+token.text,token.text))
                            if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                str1 = j.text+' '+token.text
                                amod_pairs.append(j.text+' '+token.text)
                                for k in j.lefts:
                                    if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                        str2 = k.text+' '+j.text+' '+token.text
                                        amod_pairs.append(k.text+' '+j.text+' '+token.text)
                                mtch = re.search(re.escape(str1),re.escape(str2))
                                if mtch is not None:
                                    amod_pairs.remove(str1)
                    if token.pos_ is 'VERB':
                        for j in token.lefts:
                            if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                advmod_pairs.append(j.text+' '+token.text)
                            if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                neg_pairs.append(j.text+' '+token.text)
                        for j in token.rights:
                            if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                advmod_pairs.append(token.text+' '+j.text)
                    if token.pos_ is 'ADJ':
                        for j,h in zip(token.rights,token.lefts):
                            if j.dep_ is 'xcomp' and h.dep_ is not 'neg':
                                for k in j.lefts:
                                    if k.dep_ is 'aux':
                                        xcomp_pairs.append(token.text+' '+k.text+' '+j.text)
                            elif j.dep_ is 'xcomp' and h.dep_ is 'neg':
                                if k.dep_ is 'aux':
                                        neg_pairs.append(h.text +' '+token.text+' '+k.text+' '+j.text)

            else:
                enemlist.append(enem_list)
                doc = nlp(line)
                str1=''
                str2=''
                for token in doc:
                    if token.pos_ is 'NOUN':
                        for j in token.lefts:
                            if j.dep_ == 'compound':
                                ecompound_pairs.append((j.text+' '+token.text,token.text))
                            if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                str1 = j.text+' '+token.text
                                eamod_pairs.append(j.text+' '+token.text)
                                for k in j.lefts:
                                    if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                        str2 = k.text+' '+j.text+' '+token.text
                                        eamod_pairs.append(k.text+' '+j.text+' '+token.text)
                                mtch = re.search(re.escape(str1),re.escape(str2))
                                if mtch is not None:
                                    eamod_pairs.remove(str1)
                    if token.pos_ is 'VERB':
                        for j in token.lefts:
                            if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                eadvmod_pairs.append(j.text+' '+token.text)
                            if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                eneg_pairs.append(j.text+' '+token.text)
                        for j in token.rights:
                            if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                eadvmod_pairs.append(token.text+' '+j.text)
                    if token.pos_ is 'ADJ':
                        for j in token.rights:
                            if j.dep_ is 'xcomp':
                                for k in j.lefts:
                                    if k.dep_ is 'aux':
                                        excomp_pairs.append(token.text+' '+k.text+' '+j.text)
        pairs = list(set(amod_pairs+advmod_pairs+neg_pairs+xcomp_pairs))
        epairs = list(set(eamod_pairs+eadvmod_pairs+eneg_pairs+excomp_pairs))
        for i in range(len(pairs)):
            if len(compound_pairs)!=0:
                for comp in compound_pairs:
                    mtch = re.search(re.escape(comp[1]),re.escape(pairs[i]))
                    if mtch is not None:
                        pairs[i] = pairs[i].replace(mtch.group(),comp[0])
        for i in range(len(epairs)):
            if len(ecompound_pairs)!=0:
                for comp in ecompound_pairs:
                    mtch = re.search(re.escape(comp[1]),re.escape(epairs[i]))
                    if mtch is not None:
                        epairs[i] = epairs[i].replace(mtch.group(),comp[0])

    aspect_terms.append(pairs)
    comp_terms.append(compound_pairs)
    easpect_terms.append(epairs)
    ecomp_terms.append(ecompound_pairs)
    enemy.append(enemlist)
yelp['compound_nouns'] = comp_terms
yelp['aspect_keywords'] = aspect_terms
yelp['competition'] = enemy
yelp['competition_comp_nouns'] = ecomp_terms
yelp['competition_aspects'] = easpect_terms
yelp.head()

import operator
sentiment = []
for i in range(len(yelp)):
    score_dict={'pos':0,'neg':0,'neu':0}
    if len(yelp['aspect_keywords'][i])!=0:
        for aspects in yelp['aspect_keywords'][i]:
            sent = analyser.polarity_scores(aspects)
            score_dict['neg'] += sent['neg']
            score_dict['pos'] += sent['pos']
        #score_dict['neu'] += sent['neu']
        sentiment.append(max(score_dict.items(), key=operator.itemgetter(1))[0])
    else:
        sentiment.append('NaN')
yelp['sentiment'] = sentiment
yelp.head()

int_sent = []
for sent in yelp['sentiment']:
    if sent is 'NaN':
        int_sent.append('NaN')
    elif sent is 'pos':
        int_sent.append('1')
    else:
        int_sent.append('0')
yelp['int_sent'] = int_sent
yelp

<>:38: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:42: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:42: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:46: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:52: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:54: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:54: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:56: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:56: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:59: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:59: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:61: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:63: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:63: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:65: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:67: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:6

,Review,Output,compound_nouns,aspect_keywords,competition,competition_comp_nouns,competition_aspects,sentiment,int_sent
0,Wow... Loved this place.,1,[],[],[],[],[],NaN,NaN
1,Crust is not good.,0,[],[],[],[],[],NaN,NaN
2,Not tasty and the texture was just nasty.,0,[],[],[],[],[],NaN,NaN
3,Stopped by during the late May bank holiday of...,1,"[(May bank, bank), (bank holiday, holiday), (S...",[late bank holiday],[],[],[],pos,1
4,The selection on the menu was great and so wer...,1,[],[],[],[],[],NaN,NaN
...,...,...,...,...,...,...,...,...,...
995,I think food should have flavor and texture an...,0,[],[],[],[],[],NaN,NaN
996,Appetite instantly gone.,0,[],[instantly gone],[],[],[],pos,1
997,Overall I was not impressed and would not go b...,0,[],[go back],[],[],[],pos,1
998,"The whole experience was underwhelming, and I ...",0,[],"[next time, whole experience]",[],[],[],pos,1


In [42]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

text_db = ['worst of all',
           'good but not'
]

vec = TfidfVectorizer()
tf_idf = vec.fit_transform(text_db)

# Retrieve feature names from TfidfVectorizer
feature_names = vec.get_feature_names_out()

# Create a DataFrame with TF-IDF values and feature names
tfidf_df = pd.DataFrame(tf_idf.toarray(), columns=feature_names)
print(tfidf_df)


       all      but     good      not       of    worst
0  0.57735  0.00000  0.00000  0.00000  0.57735  0.57735
1  0.00000  0.57735  0.57735  0.57735  0.00000  0.00000


In [45]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import csv

data = pd.read_csv("yelp_labelled.txt", on_bad_lines='skip')
vectorizer = TfidfVectorizer()
matrix1 = vectorizer.fit_transform(data['Review'].values.astype(str))
print(matrix1)


  (0, 957)	0.35712215818967197
  (0, 1307)	0.3278592684092347
  (0, 762)	0.5806522270296413
  (0, 1465)	0.6540757261155854
  (1, 561)	0.4114036358132431
  (1, 857)	0.3902966755949674
  (1, 691)	0.3445322012626468
  (1, 320)	0.7481397701982406
  (2, 838)	0.5014080810560408
  (2, 706)	0.3629771542240764
  (2, 1416)	0.19184518944625586
  (2, 1284)	0.5014080810560408
  (2, 1290)	0.15328780010537876
  (2, 51)	0.17900784380761256
  (2, 1273)	0.4451223417889068
  (2, 857)	0.27376636816030064
  (3, 693)	0.1416904386232428
  (3, 1037)	0.275212326506604
  (3, 1216)	0.3061063823427738
  (3, 1071)	0.3061063823427738
  (3, 867)	0.25026985846494987
  (3, 636)	0.3061063823427738
  (3, 111)	0.3061063823427738
  (3, 786)	0.26526666196792204
  (3, 720)	0.3061063823427738
  :	:
  (702, 216)	0.4155214225862832
  (702, 1277)	0.5095866973941613
  (702, 1475)	0.37178356058156486
  (702, 1416)	0.2022846256833856
  (703, 1149)	0.37742523368061703
  (703, 504)	0.34321921918741544
  (703, 715)	0.3950095430440538

In [46]:
!pip install vaderSentiment

In [47]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))

In [48]:
sentiment_analyzer_scores("The phone is super cool.")

The phone is super cool.---------------- {'neg': 0.0, 'neu': 0.326, 'pos': 0.674, 'compound': 0.7351}


In [49]:
sentiment_analyzer_scores("This car is very comfortable")

This car is very comfortable------------ {'neg': 0.0, 'neu': 0.527, 'pos': 0.473, 'compound': 0.5563}


In [50]:
sentiment_analyzer_scores("This car is not very comfortable")

This car is not very comfortable-------- {'neg': 0.369, 'neu': 0.631, 'pos': 0.0, 'compound': -0.4439}
